# Chatbot

## https://moncoachdata.com/blog/construire-un-chatbot-en-python/

In [1]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# utilisation d'un dictionnaire pour représenter un fichier JSON d'intentions
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?"],
              "responses": ["Salut à toi!", "Hello", "Comment vas tu?", "Salutations!", "Enchanté"],
             },
             {"tag": "age",
              "patterns": ["Quel âge as-tu?", "C'est quand ton anniversaire?", "Quand es-tu né?"],
              "responses": ["J'ai 25 ans", "Je suis né en 1996", 
                            "Ma date d'anniversaire est le 3 juillet et je suis né en 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["Que fais-tu ce week-end?", "Tu veux qu'on fasse un truc ensemble?", 
                           "Quels sont tes plans pour cette semaine"],
              "responses": ["Je suis libre toute la semaine", "Je n'ai rien de prévu", "Je ne suis pas occupé"]
             },
             {"tag": "name",
              "patterns": ["Quel est ton prénom?", "Comment tu t'appelles?", "Qui es-tu?"],
              "responses": ["Mon prénom est Miki", "Je suis Miki", "Miki"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "Salut", "see ya", "adios", "cya"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!"]
             }
]}

In [3]:
# initialisation de lemmatizer pour obtenir la racine des mots
lemmatizer = WordNetLemmatizer()
# création des listes
words = [] # mots des patterns
classes = [] # tags
doc_X = [] # patterns
doc_y = [] # tags dupliqués en fonction du nombre de patterns par tag
# parcourir avec une boucle For toutes les intentions
# tokéniser chaque pattern et ajouter les tokens à la liste words, les patterns et
# le tag associé à l'intention sont ajoutés aux listes correspondantes
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    # ajouter le tag aux classes s'il n'est pas déjà là 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatiser tous les mots du vocabulaire et les convertir en minuscule
# si les mots n'apparaissent pas dans la ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# trier le vocabulaire et les classes par ordre alphabétique et prendre le
# set pour s'assurer qu'il n'y a pas de doublons
classes = sorted(set(classes))
print("words", words)
print("classes", classes)
print("doc_X", doc_X)
print("doc_y", doc_y)
      

words ['hello', 'la', 'forme', 'yo', 'salut', 'ça', 'roule', 'quel', 'âge', 'as-tu', "c'est", 'quand', 'ton', 'anniversaire', 'quand', 'es-tu', 'né', 'que', 'fais-tu', 'ce', 'week-end', 'tu', 'veux', "qu'on", 'fasse', 'un', 'truc', 'ensemble', 'quels', 'sont', 'te', 'plan', 'pour', 'cette', 'semaine', 'quel', 'est', 'ton', 'prénom', 'comment', 'tu', "t'appelles", 'qui', 'es-tu', 'bye', 'salut', 'see', 'ya', 'adios', 'cya']
classes ['age', 'date', 'goodbye', 'greeting', 'name']
doc_X ['Hello', 'La forme?', 'yo', 'Salut', 'ça roule?', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'Quel est ton prénom?', "Comment tu t'appelles?", 'Qui es-tu?', 'bye', 'Salut', 'see ya', 'adios', 'cya']
doc_y ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'go

In [4]:
# liste pour les données d'entraînement
training = []
out_empty = [0] * len(classes)
# création du modèle d'ensemble de mots idx for index and doc for a word
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # marque l'index de la classe à laquelle le pattern atguel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # ajoute le one hot encoded BoW et les classes associées à la liste training
    training.append([bow, output_row])
# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)
# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [5]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

In [6]:
# modèle Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 15,109
Trainable params: 15,109
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# entraînement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 1.7479 - accuracy: 0.2105
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 1.4884 - accuracy: 0.2632
Epoch 3/200
1/1 [==============================] - 0s 500us/step - loss: 1.3563 - accuracy: 0.5789
Epoch 4/200
1/1 [==============================] - 0s 500us/step - loss: 1.3098 - accuracy: 0.5789
Epoch 5/200
1/1 [==============================] - 0s 1000us/step - loss: 1.1773 - accuracy: 0.7895
Epoch 6/200
1/1 [==============================] - 0s 999us/step - loss: 1.0091 - accuracy: 0.9474
Epoch 7/200
1/1 [==============================] - 0s 499us/step - loss: 1.0137 - accuracy: 0.7368
Epoch 8/200
1/1 [==============================] - 0s 500us/step - loss: 0.8881 - accuracy: 0.8421
Epoch 9/200
1/1 [==============================] - 0s 500us/step - loss: 0.6359 - accuracy: 0.8421
Epoch 10/200
1/1 [==============================] - 0s 507us/step - loss: 0.6324 - accuracy: 0.8947
Epoch 11/200

1/1 [==============================] - 0s 500us/step - loss: 0.0784 - accuracy: 0.9474
Epoch 84/200
1/1 [==============================] - 0s 500us/step - loss: 0.1878 - accuracy: 0.8947
Epoch 85/200
1/1 [==============================] - 0s 506us/step - loss: 0.0260 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 501us/step - loss: 0.0881 - accuracy: 0.9474
Epoch 87/200
1/1 [==============================] - 0s 501us/step - loss: 0.1303 - accuracy: 0.9474
Epoch 88/200
1/1 [==============================] - 0s 506us/step - loss: 0.1009 - accuracy: 0.8947
Epoch 89/200
1/1 [==============================] - 0s 506us/step - loss: 0.0536 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 506us/step - loss: 0.0295 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 500us/step - loss: 0.0806 - accuracy: 0.9474
Epoch 92/200
1/1 [==============================] - 0s 500us/step - loss: 0.0521 - accuracy: 1.0000
Epoch 93/200


1/1 [==============================] - 0s 499us/step - loss: 0.0737 - accuracy: 0.9474
Epoch 165/200
1/1 [==============================] - 0s 500us/step - loss: 0.1036 - accuracy: 0.9474
Epoch 166/200
1/1 [==============================] - 0s 500us/step - loss: 0.1215 - accuracy: 0.8947
Epoch 167/200
1/1 [==============================] - 0s 500us/step - loss: 0.1293 - accuracy: 0.9474
Epoch 168/200
1/1 [==============================] - 0s 500us/step - loss: 0.0724 - accuracy: 0.9474
Epoch 169/200
1/1 [==============================] - 0s 500us/step - loss: 0.0668 - accuracy: 0.9474
Epoch 170/200
1/1 [==============================] - 0s 500us/step - loss: 0.0996 - accuracy: 0.9474
Epoch 171/200
1/1 [==============================] - 0s 506us/step - loss: 0.0612 - accuracy: 0.9474
Epoch 172/200
1/1 [==============================] - 0s 500us/step - loss: 0.1005 - accuracy: 0.9474
Epoch 173/200
1/1 [==============================] - 0s 500us/step - loss: 0.1044 - accuracy: 0.8947
Epoc

In [9]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)
def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# lancement du chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

bonjour
Comment vas tu?
bonjour
Salut à toi!
bonjour
Salutations!
